In [1]:
resume_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\ganeshrajk_resume_Sept.pdf"
json_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\questions.json"

In [ ]:
import json
import openai
import PyPDF2

# Function to extract text from PDF file (resume)
def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Function to load JSON file (Q&A data)
def load_json(json_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

# Function to generate detailed summary using GPT-4
def generate_detailed_summary(pdf_text, json_data, openai_api_key):
    # Combine PDF text and JSON data into a detailed prompt
    prompt = f"""
    You are provided with a person's resume and their answers to questions from a job application. 
    Please extract the following details:
    - Experience with each skillset mentioned in the resume or Q&A.
    - The person's location, phone number, and email address.
    - The person's relocation preference, if mentioned.
    - The person's gender, if indicated.
    - Any other details relevant to a job application.

    PDF (Resume) Content:
    {pdf_text}

    JSON (Q&A) Data:
    {json.dumps(json_data, indent=2)}
    """
    
    # Set up OpenAI GPT-4 API call
    openai.api_key = openai_api_key
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes job applicant profiles."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000  # Adjust based on expected response size
    )
    
    # Extract and return the generated summary
    return response['choices'][0]['message']['content']

# Main function to process PDF, JSON, and generate the detailed summary
def main(pdf_file_path, json_file_path, openai_api_key, output_file_path="resume_detailed.txt"):
    # Extract text from the PDF (resume)
    pdf_text = extract_text_from_pdf(pdf_file_path)
    
    # Load the Q&A data from the JSON file
    json_data = load_json(json_file_path)
    
    # Generate a detailed profile summary using GPT-4
    detailed_summary = generate_detailed_summary(pdf_text, json_data, openai_api_key)
    
    # Save the generated summary to a text file
    with open(output_file_path, 'w') as file:
        file.write(detailed_summary)
    
    print(f"Detailed summary saved to {output_file_path}")

# Example usage
if __name__ == "__main__":
    pdf_file = resume_path  # Path to the resume PDF file
    json_file = json_path   # Path to the Q&A JSON file
    openai_api_key = "your_openai_api_key"  # Replace with your OpenAI API key
    
    # Generate and save the summary
    main(pdf_file, json_file, openai_api_key)